## UAP Generate & Evaluate (whitebox)

attacks
- sgd-uap (untargeted) eps = 2-12

dataset
- cifar10
- svhn

model_name
- resnet18
- [soft-pruning]
    - resnet18_sfP
    - resnet18_sfP-mixup
    - resnet18_sfP-cutout
- [post-training pruning]
    - resnet18_PX_0.Y
        - X = 2, 3, 4
        - Y = 3, 6, 9

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import json

from utils_attack import uap_batch
from utils_model import *

dir_results = './results/'
dir_uap = './uaps/'

In [2]:
nb_epoch = 10
atk_name = 'sgd-uap'
y_target = -1

In [3]:
model_names =  ['resnet18', 'resnet18_sfP', 'resnet18_sfP-mixup', 'resnet18_sfP-cutout']
for layer in [2, 3, 4]:
    for prune_pct in ['0.3', '0.6', '0.9']:
        model_names.append('resnet18_P%i-%s' % (layer, prune_pct))
model_names

['resnet18',
 'resnet18_sfP',
 'resnet18_sfP-mixup',
 'resnet18_sfP-cutout',
 'resnet18_P2-0.3',
 'resnet18_P2-0.6',
 'resnet18_P2-0.9',
 'resnet18_P3-0.3',
 'resnet18_P3-0.6',
 'resnet18_P3-0.9',
 'resnet18_P4-0.3',
 'resnet18_P4-0.6',
 'resnet18_P4-0.9']

In [4]:
def eval_uap(uap):    
    top, top_probs, top1acc, top5acc, outputs, labels = evaluate(model, testloader, uap = uap)

    print('Top 1 accuracy', sum(top1acc) / len(labels))
    print('Top 5 accuracy', sum(top5acc) / len(labels))

    dict_entry = {}
    dict_entry['model_name'] = model_name
    dict_entry['dataset'] = dataset

    dict_entry['atk_name'] = atk_name
    dict_entry['atk_param'] = y_target
    dict_entry['eps'] = eps
    if atk_name[:3] == 'sgd': dict_entry['nb_epoch'] = nb_epoch

    dict_entry['top1acc'] = sum(top1acc) / len(labels)
    dict_entry['top5acc'] = sum(top5acc) / len(labels)
    dict_entry['UER'] = 1 - sum(top1acc) / len(labels)
    print('UER ', dict_entry['UER'])
    print('\n')
    if y_target >= 0: dict_entry['tgt_success'] = sum(outputs == y_target) / len(labels)

    # Output distribution
    for i in range(10):
        dict_entry['label_dist%i' % i] = sum(outputs == i) / len(labels)

    all_results.append(dict_entry)

In [7]:
all_results = []
dataset = 'svhn'
batch_size = 1627

testloader = get_testdata(dataset, batch_size)

for model_name in model_names:
    model = get_model(model_name, dataset, batch_size)
    model.eval()

    for eps in range(2, 13, 2):
        uap_save_path = dir_uap + dataset + '/' + model_name
        print(uap_save_path + '/sgd-eps%i.pth' % eps)
        uap = torch.load(uap_save_path + '/sgd-eps%i.pth' % eps)
        eval_uap(uap)

    pd.DataFrame(all_results).to_csv(dir_results + 'eval_whitebox-%s.csv' % dataset)

Using downloaded and verified file: ./data/svhn/test_32x32.mat
Loading svhn/resnet18.pth
./uaps/svhn/resnet18/sgd-eps2.pth
 Completed: 0.00%
Top 1 accuracy 0.9303549477566072
Top 5 accuracy 0.9934311616472035
UER  0.06964505224339279


./uaps/svhn/resnet18/sgd-eps4.pth
 Completed: 0.00%
Top 1 accuracy 0.8367393976644131
Top 5 accuracy 0.9795251997541488
UER  0.16326060233558692


./uaps/svhn/resnet18/sgd-eps6.pth
 Completed: 0.00%
Top 1 accuracy 0.667178856791641
Top 5 accuracy 0.9275507068223725
UER  0.332821143208359


./uaps/svhn/resnet18/sgd-eps8.pth
 Completed: 0.00%
Top 1 accuracy 0.5442148125384143
Top 5 accuracy 0.8760755992624463
UER  0.4557851874615857


./uaps/svhn/resnet18/sgd-eps10.pth
 Completed: 0.00%
Top 1 accuracy 0.41037953288260604
Top 5 accuracy 0.7894130301167793
UER  0.5896204671173939


./uaps/svhn/resnet18/sgd-eps12.pth
 Completed: 0.00%
Top 1 accuracy 0.3676628764597419
Top 5 accuracy 0.7935233558696988
UER  0.6323371235402582


Loading svhn/resnet18_sfP.pth
Fi

./uaps/svhn/resnet18_P3-0.3/sgd-eps6.pth
 Completed: 0.00%
Top 1 accuracy 0.7032498463429625
Top 5 accuracy 0.9615473263675476
UER  0.2967501536570375


./uaps/svhn/resnet18_P3-0.3/sgd-eps8.pth
 Completed: 0.00%
Top 1 accuracy 0.5013444990780578
Top 5 accuracy 0.847380147510756
UER  0.4986555009219422


./uaps/svhn/resnet18_P3-0.3/sgd-eps10.pth
 Completed: 0.00%
Top 1 accuracy 0.4484864781807007
Top 5 accuracy 0.808658574062692
UER  0.5515135218192992


./uaps/svhn/resnet18_P3-0.3/sgd-eps12.pth
 Completed: 0.00%
Top 1 accuracy 0.3554087277197296
Top 5 accuracy 0.7444299323909035
UER  0.6445912722802705


Loading svhn/resnet18_P3-0.6.pth
./uaps/svhn/resnet18_P3-0.6/sgd-eps2.pth
 Completed: 0.00%
Top 1 accuracy 0.8946296865396435
Top 5 accuracy 0.9877074370006146
UER  0.10537031346035652


./uaps/svhn/resnet18_P3-0.6/sgd-eps4.pth
 Completed: 0.00%
Top 1 accuracy 0.7650583896742471
Top 5 accuracy 0.9721112476951445
UER  0.2349416103257529


./uaps/svhn/resnet18_P3-0.6/sgd-eps6.pth
 Comple

## View Evaluation Results

In [8]:
df = pd.read_csv(dir_results + 'eval_whitebox-svhn.csv', index_col = 0)

In [9]:
df[(df['dataset'] == 'svhn') & (df['eps'] == 8)].sort_values(by = 'UER')

,model_name,dataset,atk_name,atk_param,eps,nb_epoch,top1acc,top5acc,UER,label_dist0,label_dist1,label_dist2,label_dist3,label_dist4,label_dist5,label_dist6,label_dist7,label_dist8,label_dist9
21,resnet18_sfP-cutout,svhn,sgd-uap,-1,8,10,0.616165,0.921750,0.383835,0.036378,0.087777,0.102067,0.269361,0.055432,0.071374,0.223148,0.039682,0.072987,0.041795
15,resnet18_sfP-mixup,svhn,sgd-uap,-1,8,10,0.557314,0.900661,0.442686,0.053895,0.069415,0.085433,0.093539,0.061885,0.099455,0.427282,0.044522,0.032498,0.032076
3,resnet18,svhn,sgd-uap,-1,8,10,0.544215,0.876076,0.455785,0.036686,0.062154,0.089697,0.120813,0.061232,0.124770,0.353911,0.043831,0.064267,0.042640
27,resnet18_P2-0.3,svhn,sgd-uap,-1,8,10,0.542140,0.870928,0.457860,0.037608,0.058505,0.088545,0.145283,0.067340,0.135871,0.333167,0.041103,0.052973,0.039605
63,resnet18_P4-0.3,svhn,sgd-uap,-1,8,10,0.537223,0.862861,0.462777,0.036916,0.056315,0.092348,0.092886,0.079518,0.125269,0.384911,0.040412,0.046635,0.044791
9,resnet18_sfP,svhn,sgd-uap,-1,8,10,0.518400,0.874808,0.481600,0.039106,0.061693,0.078788,0.092655,0.050784,0.476145,0.063614,0.031769,0.042025,0.063422
69,resnet18_P4-0.6,svhn,sgd-uap,-1,8,10,0.515212,0.924746,0.484788,0.033651,0.058735,0.590158,0.069030,0.037761,0.051321,0.036071,0.045329,0.046289,0.031653
45,resnet18_P3-0.3,svhn,sgd-uap,-1,8,10,0.501344,0.847380,0.498656,0.036762,0.048440,0.088929,0.067993,0.059427,0.092348,0.475761,0.041487,0.050784,0.038069
51,resnet18_P3-0.6,svhn,sgd-uap,-1,8,10,0.460664,0.853795,0.539336,0.031692,0.052282,0.068608,0.077866,0.039144,0.087354,0.502919,0.039720,0.068954,0.031461
33,resnet18_P2-0.6,svhn,sgd-uap,-1,8,10,0.428665,0.841733,0.571335,0.053434,0.037915,0.063537,0.084665,0.030578,0.086355,0.518823,0.033766,0.056699,0.034227


In [10]:
df = pd.read_csv(dir_results + 'eval_whitebox-cifar10.csv', index_col = 0)

In [11]:
df[(df['dataset'] == 'cifar10') & (df['eps'] == 8)].sort_values(by = 'UER')

,model_name,dataset,atk_name,atk_param,eps,nb_epoch,top1acc,top5acc,UER,label_dist0,label_dist1,label_dist2,label_dist3,label_dist4,label_dist5,label_dist6,label_dist7,label_dist8,label_dist9
21,resnet18_sfP-cutout,cifar10,sgd-uap,-1,8,10,0.3634,0.9066,0.6366,0.0169,0.0953,0.0290,0.0242,0.0439,0.0113,0.6584,0.0319,0.0352,0.0539
9,resnet18_sfP,cifar10,sgd-uap,-1,8,10,0.3468,0.9457,0.6532,0.0188,0.0415,0.0274,0.0164,0.6862,0.0273,0.0424,0.0341,0.0580,0.0479
45,resnet18_P3-0.3,cifar10,sgd-uap,-1,8,10,0.3036,0.8726,0.6964,0.0259,0.0446,0.0212,0.0124,0.0176,0.0129,0.0391,0.7605,0.0166,0.0492
3,resnet18,cifar10,sgd-uap,-1,8,10,0.2932,0.9049,0.7068,0.0148,0.0390,0.0195,0.0107,0.7715,0.0166,0.0304,0.0287,0.0181,0.0507
51,resnet18_P3-0.6,cifar10,sgd-uap,-1,8,10,0.2843,0.8480,0.7157,0.0070,0.0543,0.0209,0.0153,0.0104,0.0189,0.7936,0.0211,0.0157,0.0428
69,resnet18_P4-0.6,cifar10,sgd-uap,-1,8,10,0.2820,0.8758,0.7180,0.0143,0.0426,0.0160,0.0101,0.7899,0.0185,0.0265,0.0232,0.0130,0.0459
27,resnet18_P2-0.3,cifar10,sgd-uap,-1,8,10,0.2809,0.8934,0.7191,0.0130,0.0416,0.0178,0.0096,0.7757,0.0176,0.0254,0.0362,0.0258,0.0373
63,resnet18_P4-0.3,cifar10,sgd-uap,-1,8,10,0.2732,0.8905,0.7268,0.0143,0.0389,0.0178,0.0117,0.7989,0.0164,0.0212,0.0280,0.0126,0.0402
15,resnet18_sfP-mixup,cifar10,sgd-uap,-1,8,10,0.2338,0.7686,0.7662,0.0032,0.0601,0.0036,0.0065,0.0061,0.0059,0.8540,0.0159,0.0151,0.0296
75,resnet18_P4-0.9,cifar10,sgd-uap,-1,8,10,0.2281,0.8412,0.7719,0.0076,0.0308,0.0102,0.0070,0.0104,0.0176,0.8606,0.0128,0.0077,0.0353
